<a href="https://colab.research.google.com/github/09DylanXie/Hello-and-Welcome-/blob/main/Energy_City_Software%2C_Simple_Edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import copy

class EnergyCityAssistant:
    def __init__(self, name):
        self.name = name
        # Starting Assets: 5W, 150 Gold, 5 Wood
        self.resources = {
            "Gold": 150, "Watts": 5, "Wood": 5, "Iron": 0, "Steel": 0, "Coal": 0,
            "Oil": 0, "Re-Oil": 0, "Uranium": 0, "Deuterium": 0, "Fission Cell": 0,
            "Fusion Cell": 0, "Dura-Steel": 0, "Fusion Core": 0,
            "Battery": 0, "Battery_Charge": 0
        }
        self.buildings = []
        self.utility_slots = [] # Max 3
        self.pending_watts = 0
        self.fusion_plants_built = 0
        self.shop_limits = {}
        self.battery_bought_turn = -1
        self.history = []

    def save_state(self):
        state = {
            "resources": copy.deepcopy(self.resources),
            "buildings": copy.deepcopy(self.buildings),
            "utility_slots": copy.deepcopy(self.utility_slots),
            "pending_watts": self.pending_watts,
            "fusion_plants_built": self.fusion_plants_built,
            "shop_limits": copy.deepcopy(self.shop_limits),
            "battery_bought_turn": self.battery_bought_turn
        }
        self.history.append(state)
        if len(self.history) > 10: self.history.pop(0)

    def undo(self):
        if not self.history: return "⚠️ Nothing to undo."
        state = self.history.pop()
        self.resources = state["resources"]
        self.buildings = state["buildings"]
        self.utility_slots = state["utility_slots"]
        self.pending_watts = state["pending_watts"]
        self.fusion_plants_built = state["fusion_plants_built"]
        self.shop_limits = state["shop_limits"]
        self.battery_bought_turn = state["battery_bought_turn"]
        return "↩️ Last action undone."

    def show_prices(self):
        print("\n--- PRICE REFERENCE ---")
        print("SHOP: Wood(20G), Iron(20G), Coal(40G), Steel(70G), Oil(50G), Uranium(40G), Battery(10G)")
        print("UTILS: Factory(10G+1Wd), Refinery(20G+1Ir), Mine(60G+1St)")
        print("PLANTS: Bio(20G+3Wd), Coal(30G+3Cl), Oil(50G+3Re), Fission(100G+3Fi), Fusion(150G+1Core)")
        print("REFINING (Base + 2W): Steel(4W), Dura-Steel(7W), Re-Oil(3W), Deuterium(3W), Fission Cell(4W), Fusion Cell(5W)")
        return "--- END REFERENCE ---"

    def start_turn(self, turn_num):
        self.turn_num = turn_num
        self.history = [] # Clear history to prevent undoing into previous turn
        print("\n" + "="*40 + f"\n{self.name.upper()}'S TURN | TURN: {turn_num}\n" + "="*40)

        self.resources["Watts"] += self.pending_watts
        if self.pending_watts > 0:
            print(f"⚡ Grid: +{self.pending_watts} Watts received.")
        self.pending_watts = 0

        for b in self.buildings: b['active'] = False

        mines = self.utility_slots.count("Gold Mine")
        if mines > 0:
            self.resources["Gold"] += (mines * 30)
            print(f"💰 Mines: +{mines * 30} Gold produced.")

        if turn_num > 1:
            self.resources["Gold"] += 20
            print("🎁 Allowance: +20 Gold.")

        self.shop_limits = {item: 0 for item in ["Wood", "Iron", "Coal", "Steel", "Oil", "Uranium", "Deuterium"]}

    def shop_buy(self, item, qty):
        self.save_state()
        prices = {"Wood": 20, "Iron": 20, "Coal": 40, "Steel": 70, "Oil": 50, "Uranium": 40, "Deuterium": 90, "Battery": 10}
        item = item.title()

        if item == "Battery":
            if self.resources["Battery"] > 0: return "❌ Max 1 Battery allowed ."
            if self.resources["Gold"] >= 10:
                self.resources["Gold"] -= 10
                self.resources["Battery"] = 1
                self.battery_bought_turn = self.turn_num
                return "🛒 Bought Battery."
            return "❌ Need 10 Gold."

        if item not in prices or self.shop_limits.get(item, 0) + qty > 3:
            return "❌ Shop limit (3/item) or invalid item ."

        cost = prices[item] * qty
        if self.resources["Gold"] >= cost:
            self.resources["Gold"] -= cost
            self.resources[item] += qty
            self.shop_limits[item] += qty
            return f"🛒 Bought {qty} {item}."
        return "❌ Insufficient Gold."

    def build_util(self, u_type):
        self.save_state()
        if len(self.utility_slots) >= 3: return "❌ Utility slots full ."
        u_type = u_type.lower()
        if u_type == "factory":
            if self.resources["Gold"] >= 10 and self.resources["Wood"] >= 1:
                self.resources["Gold"] -= 10; self.resources["Wood"] -= 1
                self.utility_slots.append("Resource Factory"); return "🏗️ Built Factory."
        elif u_type == "refinery":
            if self.resources["Gold"] >= 20 and self.resources["Iron"] >= 1:
                self.resources["Gold"] -= 20; self.resources["Iron"] -= 1
                self.utility_slots.append("Refinery"); return "🏗️ Built Refinery."
        elif u_type == "mine":
            if self.resources["Gold"] >= 60 and self.resources["Steel"] >= 1:
                self.resources["Gold"] -= 60; self.resources["Steel"] -= 1
                self.utility_slots.append("Gold Mine"); return "🏗️ Built Gold Mine."
        return "❌ Missing Gold/Materials ."

    def factory_produce(self, item, units):
        self.save_state()
        if "Resource Factory" not in self.utility_slots: return "❌ Need Resource Factory."
        if units < 1 or units > 6: return "❌ Factory limit 1-6 units ."
        cost = sum(range(1, units + 1)) # 1W/2W/3W...
        if self.resources["Watts"] >= cost:
            self.resources["Watts"] -= cost
            self.resources[item.title()] += units
            return f"✅ Produced {units} {item}."
        return "❌ Need more Watts."

    def power_plant(self, index):
        self.save_state()
        if index < 0 or index >= len(self.buildings): return "❌ Invalid index."
        plant = self.buildings[index]
        if plant['active']: return "❌ Already active."
        costs = {"Bio": 1, "Coal": 2, "Oil": 3, "Fission": 4, "Fusion": 5} #
        outputs = {"Bio": 2, "Coal": 4, "Oil": 7, "Fission": 10, "Fusion": 15}

        c = costs[plant['type']]
        out_val = outputs[plant['type']]
        if self.resources["Watts"] >= c:
            self.resources["Watts"] -= c; plant['active'] = True
            self.pending_watts += out_val
            return f"✅ Powered {plant['type']}! (+{out_val}W next turn)."
        return f"❌ Need {c}W to power."

    def refine(self, target):
        self.save_state()
        #
        rules = {
            "Steel": {"Iron": 1, "W": 4}, "Dura-Steel": {"Steel": 1, "W": 7},
            "Re-Oil": {"Oil": 2, "W": 3}, "Deuterium": {"Uranium": 3, "W": 3},
            "Fission Cell": {"Uranium": 2, "W": 4}, "Fusion Cell": {"Deuterium": 2, "W": 5}
        }
        target = target.title()
        if target == "Fusion Core": #
            if (self.resources["Dura-Steel"] >= 3 and self.resources["Fusion Cell"] >= 3 and
                self.resources["Watts"] >= 18 and self.resources["Gold"] >= 50):
                self.resources["Dura-Steel"] -= 3; self.resources["Fusion Cell"] -= 3
                self.resources["Watts"] -= 18; self.resources["Gold"] -= 50
                self.resources["Fusion Core"] += 1; return "⚛️ Core Synthesized!"
            return "❌ Missing components for Fusion Core."

        req = rules.get(target)
        if not req: return "❌ Invalid target."
        mat = list(req.keys())[0]
        if self.resources[mat] >= req[mat] and self.resources["Watts"] >= req["W"]:
            self.resources[mat] -= req[mat]; self.resources["Watts"] -= req["W"]
            self.resources[target] += 1; return f"✅ Refined {target}."
        return "❌ Missing Materials/Watts."

    def sell_resource(self, item, qty):
        self.save_state()
        #
        gold_items = {"Steel": 20, "Re-Oil": 20, "Fission Cell": 20, "Fusion Cell": 20}
        watt_items = ["Wood", "Iron", "Coal", "Oil"]
        item = item.title()

        if item in gold_items and self.resources.get(item, 0) >= qty:
            self.resources["Gold"] += gold_items[item] * qty
            self.resources[item] -= qty
            return f"💰 Sold for {gold_items[item]*qty} Gold."
        elif item in watt_items and self.resources.get(item, 0) >= qty:
            self.resources["Watts"] += 1 * qty
            self.resources[item] -= qty
            return f"⚡ Traded for {qty} Watts."
        return "❌ Invalid item or quantity."

    def build_plant(self, p_type):
        self.save_state()
        #
        plants = {
            "Bio": {"Gold": 20, "Mat": "Wood", "Amt": 3},
            "Coal": {"Gold": 30, "Req": "Bio", "Mat": "Coal", "Amt": 3},
            "Oil": {"Gold": 50, "Req": "Coal", "Mat": "Re-Oil", "Amt": 3},
            "Fission": {"Gold": 100, "Req": "Oil", "Mat": "Fission Cell", "Amt": 3},
            "Fusion": {"Gold": 150, "Req": "Fission", "Mat": "Fusion Core", "Amt": 1}
        }
        p_type = p_type.title()
        p = plants.get(p_type)
        if not p: return "❌ Invalid plant."
        if len(self.buildings) >= 6 and "Req" not in p: return "❌ Grid full ."

        if "Req" in p:
            idx = next((i for i, b in enumerate(self.buildings) if b['type'] == p['Req']), None)
            if idx is None: return f"❌ Need {p['Req']} Plant."
            self.resources["Gold"] += plants[p['Req']]["Gold"] # Refund
            self.buildings.pop(idx)

        if self.resources["Gold"] >= p["Gold"] and self.resources.get(p["Mat"], 0) >= p["Amt"]:
            self.resources["Gold"] -= p["Gold"]; self.resources[p["Mat"]] -= p["Amt"]
            self.buildings.append({'type': p_type, 'active': False})
            if p_type == "Fusion": self.fusion_plants_built += 1
            return f"🏗️ Built {p_type} Plant!"
        return "❌ Missing Gold/Materials."

    def show_status(self):
        print(f"\n--- {self.name.upper()}'S STATUS ---")
        print(f"💰 Gold: {self.resources['Gold']} | ⚡ Watts: {self.resources['Watts']}")
        if self.resources["Battery"] > 0:
            print(f"🔋 Battery: {self.resources['Battery_Charge']}/7W [Value: {10+10*self.resources['Battery_Charge']}G]")
        print(f"📦 Inv: { {k:v for k,v in self.resources.items() if v > 0 and 'Battery' not in k} }")
        for i, b in enumerate(self.buildings):
            print(f"  [{i}] {b['type']} Plant - {'ACTIVE' if b['active'] else 'IDLE'}")
        if self.utility_slots: print(f"🛠️ Utils: {self.utility_slots}")

def play():
    print("ENERGY CITY ASSISTANT")
    name = input("Enter your name: ")
    p = EnergyCityAssistant(name if name else "Player")
    turn = 1
    while p.fusion_plants_built < 2:
        p.start_turn(turn)
        while True:
            p.show_status()
            action = input("\nCommands: buy, build, util, power, factory, refine, sell, charge, sell_battery, prices, undo, end\n> ").lower().split()
            if not action or action[0] == "end": break
            try:
                if action[0] == "prices": print(p.show_prices())
                elif action[0] == "undo": print(p.undo())
                elif action[0] == "buy": print(p.shop_buy(action[1], int(action[2] if len(action)>2 else 1)))
                elif action[0] == "build": print(p.build_plant(action[1]))
                elif action[0] == "util": print(p.build_util(action[1]))
                elif action[0] == "power": print(p.power_plant(int(action[1])))
                elif action[0] == "factory": print(p.factory_produce(action[1], int(action[2])))
                elif action[0] == "refine": print(p.refine(" ".join(action[1:])))
                elif action[0] == "sell": print(p.sell_resource(action[1], int(action[2])))
                elif action[0] == "charge":
                    if p.resources["Battery"] == 0: print("❌ No battery."); continue
                    p.save_state()
                    amt = min(int(action[1]), 7 - p.resources["Battery_Charge"], p.resources["Watts"])
                    p.resources["Watts"] -= amt; p.resources["Battery_Charge"] += amt
                    print(f"🔋 Charged {amt}W.")
                elif action[0] == "sell_battery": print(p.sell_battery())
            except: print("⚠️ Command Error. Use 'prices' to see valid inputs.")
        turn += 1
    print(f"\n🏆 WINNER: {p.name} built 2 Fusion Plants !")

if __name__ == "__main__":
    play()